## <b>1 <span style='color:#2ae4f5'>|</span> Bibliotecas e Imports</b>

In [77]:
import os

os.environ["MPLBACKEND"] = "macosx"

import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

## <b>2 <span style='color:#2ae4f5'>|</span> Funções Auxiliares </b>

In [71]:
def bv_oracle(s: str) -> QuantumCircuit:
    n = len(s)
    qc = QuantumCircuit(n + 1)
    # para cada bit 1 em s, aplicar CNOT(control=bit i, target=ancilla)
    for i, bit in enumerate(reversed(s)):
        if bit == "1":
            qc.cx(i, n)  # n é o qubit auxiliar
    return qc


def bin_to_int(binarios):
    ints = [int(b, 2) for b in binarios]
    print(ints)

## <b>3 <span style='color:#2ae4f5'>|</span> Bernstein–Vazirani </b>

In [72]:
def bernstein_vazirani_circuit(s: str) -> QuantumCircuit:
    n = len(s)
    qc = QuantumCircuit(n + 1, n)

    # inicializa ancilla em |1> e coloca em superposição de fase
    qc.x(n)
    qc.h(n)

    qc.barrier()

    # Hadamard em todos os n qubits de dados
    qc.h(range(n))

    qc.barrier()
    # chama o oráculo
    qc.compose(bv_oracle(s), inplace=True)

    qc.barrier()
    # Hadamard novamente nos n qubits de dados
    qc.h(range(n))
    qc.barrier()

    # mede apenas os n qubits de dados
    qc.measure(range(n), range(n))

    return qc

## <b>4 <span style='color:#2ae4f5'>|</span> Análise </b>

### Simulador Ideal

#### Implementação de Teste

In [13]:
def testIdeal(secret_string: str):
    qc = bernstein_vazirani_circuit(secret_string)

    backend_ideal = AerSimulator()
    t_qc = transpile(qc, backend_ideal)

    qc.draw("mpl")
    result = backend_ideal.run(t_qc, shots=1024).result()
    counts = result.get_counts()

    print("Resultados:", counts)
    plot_histogram(counts)
    plt.title("Histogram (Ideal Simulation)")
    plt.tight_layout()
    pathSaving = "Plot_results/ideal/" + secret_string
    plt.savefig(pathSaving)

#### Testes

In [18]:
testIdeal("10000")
testIdeal("01000")
testIdeal("00100")
testIdeal("00010")
testIdeal("00001")
testIdeal("00011")
testIdeal("00111")
testIdeal("01111")
testIdeal("11111")
testIdeal("11000")
testIdeal("11100")

Resultados: {'10000': 1024}
Resultados: {'01000': 1024}
Resultados: {'00100': 1024}
Resultados: {'00010': 1024}
Resultados: {'00001': 1024}
Resultados: {'00011': 1024}
Resultados: {'00111': 1024}
Resultados: {'01111': 1024}
Resultados: {'11111': 1024}
Resultados: {'11000': 1024}
Resultados: {'11100': 1024}


### Simulador Com Ruído

#### Config

In [73]:
service = QiskitRuntimeService(channel=os.getenv("IBM_QUANTUM_SERVICE"))
backends = service.backends

qiskit_runtime_service.__init__:WARNING:2025-11-28 22:13:51,508: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: Projeto-Algo_I. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().


#### Implementação de Teste

In [ ]:
def testNoise(secret_string: str):
    qc = bernstein_vazirani_circuit(secret_string)

    for backend in backends():
        if backend.name == "ibm_marrakesh":
            continue
        print("Backend: ", backend.name)
        noise_model = NoiseModel.from_backend(backend)

        backend_noisy = AerSimulator(noise_model=noise_model)
        pass_manager = generate_preset_pass_manager(
            optimization_level=3,
            backend=backend_noisy,
            layout_method="sabre",
            routing_method="sabre",
        )
        qc_best = pass_manager.run(qc)
        result = backend_noisy.run(qc_best, shots=1024).result()
        counts = result.get_counts()
        plot_histogram(counts)
        plt.title(f"Histogram for backend {backend.name}")
        plt.tight_layout()
        pathSaving = "Plot_results/noisy/" + secret_string + "_" + backend.name
        plt.savefig(pathSaving)

#### Testes

In [ ]:
testNoise("10000")
testNoise("01000")
testNoise("00100")
testNoise("00010")
testNoise("00001")
testNoise("00011")
testNoise("00111")
testNoise("01111")
testNoise("11111")
testNoise("11000")
testNoise("11100")

qiskit_runtime_service.backends:WARNING:2025-11-28 20:41:58,895: Loading instance: Projeto-Algo_I, plan: open


Backend:  ibm_fez
Backend:  ibm_torino


qiskit_runtime_service.backends:WARNING:2025-11-28 20:42:07,562: Loading instance: Projeto-Algo_I, plan: open


Backend:  ibm_fez
Backend:  ibm_torino


qiskit_runtime_service.backends:WARNING:2025-11-28 20:42:16,185: Loading instance: Projeto-Algo_I, plan: open


...

Backend:  ibm_fez
Backend:  ibm_torino


qiskit_runtime_service.backends:WARNING:2025-11-28 20:43:23,933: Loading instance: Projeto-Algo_I, plan: open


Backend:  ibm_fez
Backend:  ibm_torino


qiskit_runtime_service.backends:WARNING:2025-11-28 20:43:32,424: Loading instance: Projeto-Algo_I, plan: open


Backend:  ibm_fez
Backend:  ibm_torino


### Processor Quantico Real

#### Config

In [86]:
service = QiskitRuntimeService()

backends = service.backends

qiskit_runtime_service.__init__:WARNING:2025-11-28 22:22:36,760: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: Projeto-Algo_I. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().


#### Implementação de Teste

In [ ]:
def testReal(secret_string: str):
    qc = bernstein_vazirani_circuit(secret_string)

    for backend in backends():
        if backend.name == "ibm_marrakesh":
            continue

        qc_transpiled = transpile(qc, backend, optimization_level=2)

        sampler = Sampler(mode=backend)
        qc_job = sampler.run([qc_transpiled], shots=1000)

        result = qc_job.result()

        bitarray = result[0].data.c  # BitArray com todos os shots
        counts = bitarray.get_counts()  # Converte para dicionário estilo Qiskit

        plot_histogram(counts)
        plt.title(f"Histogram for backend {backend.name}")
        plt.tight_layout()
        pathSaving = "Plot_results/real/" + secret_string + "_" + backend.name
        plt.savefig(pathSaving)

: 

#### Testes

In [90]:
testReal("10000")
testReal("01000")
testReal("00100")
testReal("00010")
testReal("00001")
testReal("00011")
testReal("00111")
testReal("01111")
testReal("11111")
testReal("11000")
testReal("11100")

qiskit_runtime_service.backends:WARNING:2025-11-28 22:24:21,225: Loading instance: Projeto-Algo_I, plan: open
qiskit_runtime_service.backends:WARNING:2025-11-28 22:24:30,180: Loading instance: Projeto-Algo_I, plan: open
qiskit_runtime_service.backends:WARNING:2025-11-28 22:24:38,022: Loading instance: Projeto-Algo_I, plan: open
qiskit_runtime_service.backends:WARNING:2025-11-28 22:24:45,414: Loading instance: Projeto-Algo_I, plan: open
qiskit_runtime_service.backends:WARNING:2025-11-28 22:24:52,256: Loading instance: Projeto-Algo_I, plan: open
qiskit_runtime_service.backends:WARNING:2025-11-28 22:24:58,957: Loading instance: Projeto-Algo_I, plan: open
qiskit_runtime_service.backends:WARNING:2025-11-28 22:25:05,891: Loading instance: Projeto-Algo_I, plan: open
qiskit_runtime_service.backends:WARNING:2025-11-28 22:25:12,754: Loading instance: Projeto-Algo_I, plan: open
qiskit_runtime_service.backends:WARNING:2025-11-28 22:25:20,201: Loading instance: Projeto-Algo_I, plan: open
qiskit_run